In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

f = open('twitch_secrets.txt')
secret_data = f.read()
f.close()
secret_data

PGE_Secret = secret_data.split('\n')[3]

sql = """
    SELECT user_id, tag_ids FROM twitch_api_scrape
    """

#INPUT YOUR OWN CONNECTION STRING HERE
conn_string = 'postgresql://postgres:'+ PGE_Secret +'@192.168.1.21:5432/postgres'

#perform to_sql test and print result
db = create_engine(conn_string)
conn = db.connect()

#execute query
pg_conn = psycopg2.connect(conn_string)
cur = pg_conn.cursor()
cur.execute(sql)
tags = cur.fetchall()

#Create a dataframe from the query results
tags_df = pd.DataFrame(tags, columns=['user_id', 'tag_ids'])

#Explode the tag_ids column into a list of tags
tags_df['tag_ids_arr'] = tags_df['tag_ids'].str.strip('{}').str.split(',')
tags_df_fin = tags_df.explode('tag_ids_arr').reset_index(drop=True)
tags_df_fin = tags_df_fin.drop(columns=['tag_ids'])

tags_df_fin.to_sql('twitch_tag_key', con=conn, if_exists='append', index=False)

pg_conn.commit()
cur.close()

#close connection
conn.close()